In [252]:
import pandas as pd
import numpy as np
import datetime

#api
import praw

#pre-processing
import nltk 
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import MWETokenizer 
from nltk import word_tokenize
import re
# import string
#from textblob import TextBlob
#from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import gensim
from collections import Counter
# nltk.download()

from NLP_functions import display_topics, prob_df, topic_matrix

#analysis
from sklearn.decomposition import TruncatedSVD, NMF

In [2]:
from NLP_functions import display_topics, prob_df, topic_matrix

In [3]:
reddit = praw.Reddit(client_id='oCxykEUclueAuA', client_secret='M8wVQw3nFxMsXHLGxurnTtkN-Mdvtg', user_agent='Crypto')

Version 7.0.0 of praw is outdated. Version 7.1.4 was released Sunday February 07, 2021.


#### Get top posts of all time

In [4]:
posts = []
ml_subreddit = reddit.subreddit('Cryptocurrency')

for post in list(ml_subreddit.top('all', limit=1000)):
    date = datetime.datetime.utcfromtimestamp(post.created_utc)
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, date])
    
posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'date'])
print(posts)

                                                 title  score      id  \
0    Elon Musk reminds me of a rich billionaire who...  39192  lce6zm   
1    CryptoNick is deleting all of his BitConnect v...  26496  7r0ftz   
2    Reminder: Robinhood blocked several stocks fro...  26234  lfrslo   
3    I will tell you exactly what is going on here,...  20145  7vga1y   
4    Robinhood is launching a Crypto Trading app to...  19963  7sx5ze   
..                                                 ...    ...     ...   
927  Apple Co-Founder Steve Wozniak Outlines the Pa...   1492  9rcezl   
928  PwC Hong Kong and PwC Singapore announce joint...   1487  8gy2l6   
929                                 We Are All Satoshi   1485  gqt4ze   
930             Top 101 Coins Grouped by Usage/Purpose   1487  lgeots   
931  Hackers Donate Stolen Bitcoin Worth $20,000 to...   1489  jeowcu   

          subreddit                                                url  \
0    CryptoCurrency  https://www.reddit.com/r/Cry

#### Get top coments from each top post

In [5]:
top_comments = []
for i,d in zip(posts.id, posts.date):
    try:
        submission = reddit.submission(id=i)
        submission.comments.replace_more(limit=0)
        comments=list([(comment.body) for comment in submission.comments.list()])
        comment_string = ''
        for x in comments[:20]:
            comment_string = comment_string + ' ' + x
        top_comments.append([comment_string,d])
    except:
        comments=None
top_comments = pd.DataFrame(top_comments,columns=['comment', 'date'])
print(top_comments)

                                               comment                date
0     If your life savings hinges on a Tweet from a... 2021-02-04 12:02:16
1     **Some Important Reminders:**\n\n* Even in th... 2018-01-17 11:46:08
2     It’s not even real Bitcoin.  You can never wi... 2021-02-09 01:55:15
3     [deleted] It's a shame i can vote this up onl... 2018-02-05 17:21:09
4     Reminder: all referral links are prohibited i... 2018-01-25 16:19:04
..                                                 ...                 ...
927   Not a bad supporter to have! Show me your bag... 2018-10-25 17:31:26
928   Alibaba working with PWC on a food supply cha... 2018-05-04 09:55:54
929   "I am the real Slim Shady" lol Except Craig W... 2020-05-26 08:28:25
930   Well written! Didnt know there was soo many s... 2021-02-09 22:02:05
931   Who is the true Robinhood here? > Bitcoin Wor... 2020-10-20 12:54:38

[932 rows x 2 columns]


## Cleaning

### EDA of Comments from each year to get stop words

In [10]:
# all_comments = comment_year.loc['2017', 'Comments'] #concatenate comments column into one string

# all_comments = re.sub(r'\([^)]*\)', '', all_comments) #remove links
# all_comments = re.sub('\w*\d\w*', '', all_comments) #remove numbers
# all_comments = re.sub(r'[^\w\s]', '', (all_comments))#remove punctuation
# all_comments = all_comments.lower() #lowercase
# all_comments = all_comments.replace('btc', 'bitcoin') #replace bitcoin acronym 

# #create stop words list
# stop = stopwords.words('english')
# stop += [ 'moderator', 'subreddit', 'reddit' , 'question', 'concern', 'please', 'contact', 'im', 
#         'people', 'gon', 'seems', 'something', 'year', 'like', 'mod']
# stop = set(stop)

# mwe_tokenizer = MWETokenizer([ ('bull','run'), ('market','cap'), ('pump','dump'),
#                              ('stock','market'), ('flaired', 'inaccurately'), ('get', 'rich', 'quick')])

# words = mwe_tokenizer.tokenize(word_tokenize(all_comments))  # tokenize words

# #lemmatize words
# lemmatizer = nltk.stem.WordNetLemmatizer()
# words = [(lemmatizer.lemmatize(w)) for w in words]

# words = [w for w in words if w not in stop] #remove stop words

# #create n grams
# n = 2
# bigrams = ngrams(words, n)

# #count phrases
# counter = Counter()
# counter += Counter(bigrams)

# for word, count in counter.most_common(30):
#     print('%20s %i' % (" ".join(word), count))

combine comments by year

In [6]:
top_comments['year'] = top_comments['date'].dt.year
years = ['2017', '2018', '2019', '2020', '2021']
top_comments['year'].value_counts()

2018    404
2020    200
2021    125
2019    114
2017     89
Name: year, dtype: int64

In [331]:
from nltk.corpus import wordnet
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)


In [309]:
comment_year = pd.DataFrame(index=years, columns=['Comments'])
for y in years:
    comment_string = ''
    for index, row in top_comments.iterrows():
        if str(row[2]) == y:
            comment_string = comment_string + ' ' + row[0]
            
    comment_string = re.sub(r'\([^)]*\)', '', comment_string) #remove links
    comment_string = re.sub('\w*\d\w*', '', comment_string) #remove numbers
    comment_string = re.sub(r'[^\w\s]', '', (comment_string))#remove punctuation
    comment_string = comment_string.lower() #lowercase
    comment_string = comment_string.replace('btc', 'bitcoin') #replace bitcoin acronym 
    comment_string = comment_string.replace('bch', 'bitcoin_cash') 
    comment_string = comment_string.replace('eth', 'ethereum')
    
    NON_ASCII = re.compile(r'[^a-z0-1\s]')
    comment_string = NON_ASCII.sub(r'', comment_string)
    
    #create stop words list
    stop = stopwords.words('english')
    stop += [ 'moderator', 'subreddit', 'reddit' , 'question', 'concern', 'please', 'contact', 'im', 
            'people', 'gon', 'seems', 'something', 'year', 'like', 'mod', 'wa', 'thread', 'crypto', 'get',
            'see', 'deleted']
    stop = set(stop)

    mwe_tokenizer = MWETokenizer([ ('bull','run'), ('market','cap'), ('pump','dump'), ('bitcoin','cash'),
                             ('stock','market'), ('flaired', 'inaccurately')])
    
    comment_token = mwe_tokenizer.tokenize(word_tokenize(comment_string))  # tokenize words
    
    comment_token = [w.strip() for w in comment_token] #remove spaces
    
   



    # 1. Init Lemmatizer
    lemmatizer = nltk.stem.WordNetLemmatizer()

    # 2. Lemmatize Single Word with the appropriate POS tag
    word = 'feet'
    comment_token_lemm = [lemmatizer.lemmatize(word, get_wordnet_pos(word)))

    # 3. Lemmatize a Sentence with the appropriate POS tag
    sentence = "The striped bats are hanging on their feet for best"
    print([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sentence)])
    
    #lemmatize words
#     stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
#     lemmatizer = nltk.stem.WordNetLemmatizer()
#     comment_token_lemm = [(stemmer.stem(lemmatizer.lemmatize(w)) for w in comment_token]

    comment_token_lemm = [w for w in comment_token_lemm if w not in stop] #remove stop words
    
    comment_token_lemm = " ".join(comment_token_lemm) #join back into one string
    
    comment_year.loc[y, 'Comments'] = comment_token_lemm #add to df

In [310]:
print(comment_year)

                                               Comments
2017  longer version raccoon loses two piece candyfl...
2018  important reminder even case public figure str...
2019  twitter post random guy popular giving legitim...
2020  offered cabinet position biden admin tldr form...
2021  life saving hinge tweet crazy billionaire dont...


In [311]:
# comment_year = re.sub(r'\([^)]*\)', '', all_comments) #remove links
# comment_year = re.sub('\w*\d\w*', '', all_comments) #remove numbers
# comment_year = re.sub(r'[^\w\s]', '', (all_comments))#remove punctuation
# comment_year = comment_year.lower() #lowercase
# comment_year = comment_year.replace('btc', 'bitcoin') #replace bitcoin acronym 

In [312]:
# new = s.apply(fun) 

# Topic Modeling

In [313]:
comments = list(comment_year['Comments'])

topics = 2
topic_words = 20
topic_names = None

### Count Vectorization

In [314]:
cv = CountVectorizer(stop_words=stop, ngram_range=(1,3))
X_cv = cv.fit_transform(comments)
pd.DataFrame(X_cv.toarray(), columns=cv.get_feature_names())

,aa,aa ofensai,aa ofensai de,aaa,aaa anonymous,aaa anonymous altcoins,aaaaaah,aaaaaah thats,aaaaaah thats whan,aaaaand,...,zuckerbucks inevitable,zuckerbucks inevitable control,zuckercash,zuckercash stable,zuckercash stable need,zuckercash zuckercash,zuckercash zuckercash stable,zunes,zunes never,zunes never replace
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,0,0,0,0,0,0,2,...,0,0,2,1,1,1,1,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,1,1,0,0,0,0,0,1,1,1
3,0,0,0,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,1,1,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0


#### Latent Semantic Analysis (LSA) with Count Vectorization

In [315]:
lsa_cv = TruncatedSVD(topics)
X_lsa_cv = lsa_cv.fit_transform(X_cv)
lsa_cv.explained_variance_ratio_

array([0.85715558, 0.06249011])

In [316]:
display_topics(lsa_cv, cv.get_feature_names(), topic_words, topic_names)


Topic  0
bitcoin, dont, money, would, one, coin, time, ha, make, good, buy, still, market, know, going, think, thing, even, go, really

Topic  1
bitcoin, buy, ethereum, doge, time, bought, never, moon, xrp, gif, paypal, buying, defi, pump, alt, sell, meme, sold, always, next


In [317]:
# topic_matrix(lsa_cv, cv.get_feature_names(), topics)

In [318]:
# prob_df(X_lsa_cv, topics)

#### Non-Negative Matrix Factorization (NMF) with Count Vectorization

In [319]:
nmf_cv = NMF(topics)
X_nmf_cv = nmf_cv.fit_transform(X_cv)
# nmf_cv.explained_variance_ratio_

In [320]:
display_topics(nmf_cv, cv.get_feature_names(), topic_words, topic_names)


Topic  0
bitcoin, dont, money, coin, would, one, ha, time, make, market, good, still, thing, know, going, think, exchange, even, buy, need

Topic  1
bitcoin, dont, time, money, buy, one, would, ha, good, coin, make, know, go, think, going, even, still, much, back, day


In [321]:
# topic_matrix(nmf_cv, cv.get_feature_names(), topics)

In [322]:
prob_df(X_nmf_cv, topics)

,topic_1,topic_2
2017,5.50617,17.27875
2018,59.34721,0.00000
2019,5.17410,22.39096
2020,0.00000,50.97603
2021,1.73046,39.91785


## TF-IDF Vectorizer

In [323]:
tfid = TfidfVectorizer(stop_words= stop, ngram_range=(1,3))
X_tfid = tfid.fit_transform(comments)
pd.DataFrame(X_tfid.toarray(), columns=tfid.get_feature_names())

,aa,aa ofensai,aa ofensai de,aaa,aaa anonymous,aaa anonymous altcoins,aaaaaah,aaaaaah thats,aaaaaah thats whan,aaaaand,...,zuckerbucks inevitable,zuckerbucks inevitable control,zuckercash,zuckercash stable,zuckercash stable need,zuckercash zuckercash,zuckercash zuckercash stable,zunes,zunes never,zunes never replace
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000563,0.000563,0.000563,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000755,...,0.000000,0.000000,0.001127,0.000563,0.000563,0.000563,0.000563,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001268,...,0.001893,0.001893,0.000000,0.000000,0.000000,0.000000,0.000000,0.001893,0.001893,0.001893
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001303,0.001303,0.001303,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.001481,0.001481,0.001481,0.000000,0.000000,0.000000,0.001984,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


#### Latent Semantic Analysis (LSA) with TF-IDF Vectorization 

In [324]:
lsa_tfid = TruncatedSVD(topics)
X_lsa_tfid = lsa_tfid.fit_transform(X_tfid)
lsa_tfid.explained_variance_ratio_

array([0.0029449 , 0.33373762])

In [325]:
display_topics(lsa_tfid, tfid.get_feature_names(), topic_words, topic_names)


Topic  0
bitcoin, dont, money, time, would, one, coin, ha, buy, make, good, know, think, going, go, still, even, thing, really, much

Topic  1
coin, word word, word, price, bitcoincash, word word word, ripple, litecoin, buy, compressed commented, today faq full, full list source, today faq, let know compressed, faq full, compressed commented today, know compressed, faq full list, know compressed commented, full list


In [326]:
# topic_matrix(lsa_tfid, tfid.get_feature_names(), topics)

In [327]:
# prob_df(X_lsa_tfid, topics)

#### Non-Negative Matrix Factorization (NMF) with TF-IDF Vectorization 

In [328]:
nmf_tfid = NMF(topics)
X_nmf_tfid = nmf_tfid.fit_transform(X_tfid)

In [329]:
display_topics(nmf_tfid, tfid.get_feature_names(), topic_words, topic_names)


Topic  0
bitcoin, dont, money, time, one, would, ha, buy, make, coin, good, think, know, still, go, going, even, thing, market, really

Topic  1
bitcoin, coin, would, dont, time, buy, money, ha, one, price, good, much, day, make, going, also, know, really, want, go


In [219]:
# topic_matrix(nmf_tfid, tfid.get_feature_names(), topics)

In [250]:
prob_df(X_nmf_tfid, topics)

,topic_1,topic_2
2017,0.00000,1.2536
2018,0.60487,0.1656
2019,0.66187,0.0000
2020,0.67885,0.0000
2021,0.66393,0.0000


## Word2Vec

In [281]:
tokenized_comments = [gensim.utils.simple_preprocess(d) for d in comments]

# size refers to the desired dimension of our word vectors
# window refers to the size of our context window
# sg means that we are using the Skip-gram architecture

word2vec = gensim.models.Word2Vec(tokenized_comments, size=10, window=2,min_count=1)

# Path to where the word2vec file lives
google_vec_file = '/Users/racheldilley/Downloads/GoogleNews-vectors-negative300.bin'

# Load it!  This might take a few minutes...
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format(google_vec_file, binary=True)

In [292]:
word2vec.most_similar('doge')

<ipython-input-292-9a34b3f00e64>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  word2vec.most_similar('doge')


[('wallet', 0.9982737302780151),
 ('back', 0.9982404708862305),
 ('point', 0.9979836940765381),
 ('deleted', 0.9978646039962769),
 ('way', 0.9977639317512512),
 ('going', 0.9977407455444336),
 ('news', 0.9975132942199707),
 ('bitcoin', 0.9974022507667542),
 ('got', 0.9972648620605469),
 ('week', 0.997062087059021)]

In [290]:
print(word2vec.train(comments, total_examples=word2vec.corpus_count, epochs=word2vec.epochs))

(0, 3213660)


In [288]:
word2vec.wv

In [274]:
class WordVecVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = 300
    def fit(self, X, y):
        return self
    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in texts.split() if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for texts in X
        ])

wtv = WordVecVectorizer(word2vec_model)
X_wtv = wtv.transform(comments)
print(X_wtv)

[[ 0.03869801  0.01764409 -0.00234458 ... -0.05396579  0.0022748
  -0.02506784]
 [ 0.03272341  0.01611841  0.00059652 ... -0.05530431  0.0008333
  -0.02268457]
 [ 0.0324393   0.01325334  0.0021516  ... -0.05470357  0.00699584
  -0.02178209]
 [ 0.03595109  0.01757328  0.00103821 ... -0.05560936  0.00473393
  -0.02174076]
 [ 0.03675701  0.02153272 -0.00259163 ... -0.05126861  0.00753019
  -0.02464241]]


In [275]:
print(word2vec_model.wv.vocab.keys())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

<ipython-input-275-0b4962db2f99>:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  print(word2vec_model.wv.vocab.keys())


In [266]:
lsa_wtv = TruncatedSVD(topics)
X_lsa_wtv = lsa_tfid.fit_transform(X_wtv)
# lsa_wtv.explained_variance_ratio_

In [276]:
display_topics(lsa_wtv, word2vec_model.wv.vocab.keys(), topic_words, topic_names)

<ipython-input-276-9addc4b75dba>:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  display_topics(lsa_wtv, word2vec_model.wv.vocab.keys(), topic_words, topic_names)


AttributeError: 'TruncatedSVD' object has no attribute 'components_'

## GloVe

In [263]:
# from sklearn.cluster import KMeans
# km = KMeans(
#     n_clusters=2, init='random',
#     n_init=10, max_iter=300, 
#     tol=1e-04, random_state=0
# )
# y_km = km.fit_predict(X_wtv)
# df = pd.DataFrame({'year' :comments, 'topic_cluster' :y_km })
# df

In [ ]:
# from sklearn.manifold import TSNE
# model = TSNE(n_components=2, perplexity=50, learning_rate=100, 
#                         n_iter=1000, verbose=1, random_state=0, angle=0.75)
# tsne_features = model.fit_transform(lda_matrix)
# df = pd.DataFrame(tsne_features)
# df['topic'] = lda_matrix.argmax(axis=1)
# df.columns = ['TSNE1', 'TSNE2', 'topic']
# import seaborn as sns
# plt.figure(figsize=(15, 10))
# plt.title('T-SNE plot of different headlines ( headlines are clustered among their topics)')
# ax = sns.scatterplot(x = 'TSNE1', y = 'TSNE2', hue = 'topic', data = df, legend = 'full')
# plt.show()

In [ ]:
# def get_top_n_words(corpus, n=10):
#   vec = CountVectorizer(stop_words='english').fit(corpus)
#   bag_of_words = vec.transform(corpus)
#   sum_words = bag_of_words.sum(axis=0) 
#   words_freq = [(word, sum_words[0, idx]) for word, idx in   vec.vocabulary_.items()]
#   words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
#   return words_freq[:n]
# words = []
# word_values = []
# for i,j in get_top_n_words(headlines['headline_text'],15):
#   words.append(i)
#   word_values.append(j)
# fig, ax = plt.subplots(figsize=(16,8))
# ax.bar(range(len(words)), word_values);
# ax.set_xticks(range(len(words)));
# ax.set_xticklabels(words, rotation='vertical');
# ax.set_title('Top 15 words in the headlines dataset');
# ax.set_xlabel('Word');
# ax.set_ylabel('Number of occurences');
# plt.show()


In [ ]:
# from wordcloud import WordCloud
# all_words = ''.join([word for word in headlines['headline_text'][0:100000]])
# all_words
# wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(all_words)
# plt.figure(figsize=(15, 8))
# plt.imshow(wordcloud, interpolation="bilinear")
# plt.axis('off')
# plt.title("Some frequent words used in the headlines", weight='bold', fontsize=14)
# plt.show()